In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2

print("open cv2 version: %s" % cv2.__version__)

open cv2 version: 3.3.1


In [2]:
def load_imgs(vidPath, firstFrame, lastFrame):
    '''
    Reads frames from video and stores as a list
    
    Parameters: 
    vidPath (string): Path to video
    
    Returns: 
    list of images
    '''
    
    cap = cv2.VideoCapture(vidPath)
    length = np.arange(start, stop)
    print(len(length)) 
    imgs = []
    for ff in length:
        cap.set(1,ff)
        ret, frame = cap.read()
        if np.mod(ff, 100) == 0:
            print(ff) # prints progress in 50 frames

            # convert to grey
            #img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        imgs.append(frame)
#         print("length of imgs: %s" % len(imgs))
            
    cap.release()
    return imgs

In [3]:
# adjust gamma, if your vid is too dark
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [4]:
#give path to videofiles
vidpath = r"G:\My Drive\Mahad\Light Level Experiments\Videos Used for Analysis/"
export_path = r'./dataFolder/Output/Proboscis/TrackedVideos/'

# the Ref provides the start and stop frames
path_frames = glob.glob(r"./dataFolders/Output/Step5_v3/" + '*.csv')

In [ ]:
videosTobeMade = pd.read_csv('./dataFolders/Output/Proboscis/ListOFVideosManuallyTracked.csv')

In [ ]:
videosTobeMade.head()

In [18]:
videosTobeMade = pd.DataFrame()
videosTobeMade.loc[0, 'name'] = 'L50_c-3_m9_FirstVisit'

In [19]:
# videosTobeMade = pd.DataFrame()
videosTobeMade

,name
0,L50_c-3_m9_FirstVisit


In [20]:
visitnum = ['FirstVisit', 'Lastvisit']

for name in videosTobeMade.name:
    mothID = name.split('_')[0] + '_' + name.split('_')[1] + '_' + name.split('_')[2]

    # get the video section
    videoPath = vidpath + mothID + '.mp4'
    frame_ref = [f for f in path_frames if mothID + '_' in f][0]
    file = pd.read_csv(frame_ref)

    visit = name.split('_')[3]
    if visit == 'FirstVisit':
        row = 0
    else:
        row = -1

    select = file.iloc[row, :]
    start = int(select['In_Frame'] + 1)
    stop = int(select['Out_Frame'])

    # get vid info
    cap = cv2.VideoCapture(videoPath)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    print(length, width, height, fps)
    # load images
    imList = load_imgs(videoPath, start, stop)
    cap.release()

    print("length of section you want to read: %s" % int(stop - start))
    print("length of the image list: %s" % len(imList))
    vidLen = len(imList)

    # make directory to store images
    tempImgDirectory = os.path.join(os.path.dirname(r"./dataFolders/Output/Proboscis/ManualTracking/"), 
                                    "tmpImgs")
    if not os.path.exists(tempImgDirectory):
        os.mkdir(tempImgDirectory)

    #######################################################
    outputImgs = []
    for frameNum in np.arange(0, vidLen):

        # adjust gamma
        image = adjust_gamma(imList[frameNum], 1.5)           
        # save img
        cv2.imwrite(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"),
                   image)           
        if np.mod(frameNum, 100) == 0:
            print(frameNum, " of ", vidLen)
    #################################################################################

    # convert images to video with ffmpeg
    returnDir = os.getcwd()
    os.chdir(tempImgDirectory)
    print(tempImgDirectory)

    # use ffmpeg to convert directory to video
    # -r is output frame rate
    os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')

    nameofVideo = r'./' + name + '.mp4'
    os.rename(r'./outputVid.mp4', nameofVideo)
    os.chdir(returnDir)

    # delete images from directory
    delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]
    dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

145118 648 488 30.0
1817
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
length of section you want to read: 1817
length of the image list: 1817
0  of  1817
100  of  1817
200  of  1817
300  of  1817
400  of  1817
500  of  1817
600  of  1817
700  of  1817
800  of  1817
900  of  1817
1000  of  1817
1100  of  1817
1200  of  1817
1300  of  1817
1400  of  1817
1500  of  1817
1600  of  1817
1700  of  1817
1800  of  1817
./dataFolders/Output/Proboscis/ManualTracking\tmpImgs


In [ ]:
os.chdir('../../')
os.getcwd()